# Writing data to and reading data from a Database using Python

## Libraries and settings

In [51]:
# Libraries
import os
import sqlite3
import fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Function to close a sqlite db-connection
def check_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False

# Get current working directory
print(os.getcwd())

/workspaces/data_analytics/Week_02


## Create sqlite data base

In [52]:
# Create data base
conn = sqlite3.connect('apartment_database.db') 
cursor = conn.cursor()

# Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'), '*.db')
for i in flist:
    print(i)

supermarkets_database.db
apartment_database.db


## Create SQL-table in the database

In [53]:
cursor.execute('''CREATE TABLE IF NOT EXISTS apartments_table (OrderId VARCHAR(50),
                                                               Adress VARCHAR(200),
                                                               Rooms DECIMAL(8,2),
                                                               Area INT(8),
                                                               Price DECIMAL(8,2))''')
# Confirm changes to the table
conn.commit()

## Read data from file to data frame

In [54]:
df = pd.read_csv('apartments_data_prepared.csv',
                  sep=',', 
                  encoding='utf-8')[['web-scraper-order', 'address_raw', 'rooms', 'area', 'price']]
print(df.shape)
df.head(5)

(1008, 5)


,web-scraper-order,address_raw,rooms,area,price
0,1693998201-1,"Neuhusstrasse 6, 8630 Rüti ZH, ZH",3.0,49.0,1441.0
1,1693998201-2,"Zürcherstrasse 1, 8173 Neerach, ZH",3.5,65.0,1850.0
2,1693998201-3,"Cramerstrasse 8-12, 8004 Zürich, ZH",NaN,NaN,2686.0
3,1693998201-4,"Cramerstrasse 8-12, 8004 Zürich, ZH",2.0,54.0,4853.0
4,1693998201-5,"Rotachstrasse 33, 8003 Zürich, ZH",2.0,49.0,4335.0


## Write data to the SQL-table in data base

In [55]:
df.to_sql(name = 'apartments_table',
          con = conn,
          index = False,
          if_exists = 'replace')

1008

## Query the SQL-table

In [56]:
# Query the SQL-table
cursor.execute('''SELECT *
               FROM apartments_table
               WHERE rooms >=1''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['OrderId','Address','Rooms','Area','Price'])    
df

,OrderId,Address,Rooms,Area,Price
0,1693998201-1,"Neuhusstrasse 6, 8630 Rüti ZH, ZH",3.0,49.0,1441.0
1,1693998201-2,"Zürcherstrasse 1, 8173 Neerach, ZH",3.5,65.0,1850.0
2,1693998201-4,"Cramerstrasse 8-12, 8004 Zürich, ZH",2.0,54.0,4853.0
3,1693998201-5,"Rotachstrasse 33, 8003 Zürich, ZH",2.0,49.0,4335.0
4,1693998201-16,"Wolframplatz 1, 8045 Zürich, ZH",2.0,32.0,3515.0
...,...,...,...,...,...
958,1693998381-1004,"Hölderlinstrasse 12, 8032 Zürich, ZH",3.5,82.0,2830.0
959,1693998381-1005,"Hegifeldstrasse 70, 8404 Reutlingen (Winterthu...",4.0,73.0,1713.0
960,1693998381-1006,"Holzmoosrütisteig 2b, 8820 Wädenswil, ZH",4.5,110.0,3500.0
961,1693998381-1007,"Vogelbuckstrasse 25, 8307 Effretikon, ZH",3.0,68.0,1650.0


# **Additional SQL-queries**

In [57]:
##4c) Filter all apartments with >= 4.0 rooms and where the area is >= 100m2

# Query the SQL-table
conn = sqlite3.connect('apartment_database.db') 

cursor = conn.cursor()

cursor.execute('''SELECT *
               FROM apartments_table
               WHERE rooms >=4.0
               AND area >= 100''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['OrderId','Address','Rooms','Area','Price'])    

conn.close()
df.head()

,OrderId,Address,Rooms,Area,Price
0,1693998205-26,"Frowiesstrasse 36, 8344 Bäretswil, ZH",5.5,140.0,2750.0
1,1693998205-27,"Kanzleistrasse 15, 8418 Schlatt ZH, ZH",4.5,104.0,2700.0
2,1693998205-29,"Bolletweg 14, 8934 Knonau, ZH",5.5,150.0,3390.0
3,1693998205-31,"Adlergass 6, 8805 Richterswil, ZH",5.5,165.0,NaN
4,1693998205-33,"Haldenstrasse 67, 8602 Wangen b. Dübendorf, ZH",4.5,118.0,3390.0


In [58]:
#4d) Write an SQL-query to calculate the average price per room size

conn = sqlite3.connect('apartment_database.db') 

cursor = conn.cursor()

cursor.execute('''SELECT rooms, AVG(price)    
               FROM apartments_table    
               GROUP BY rooms''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['Rooms','Avg_Price'])    

conn.close()
df


,Rooms,Avg_Price
0,NaN,3250.844444
1,1.0,1290.311475
2,1.5,2036.418605
3,2.0,2133.375000
4,2.5,2450.907407
5,3.0,2112.017857
6,3.5,2643.370968
7,4.0,2890.257143
8,4.5,3029.432990
9,5.0,2993.333333


In [59]:
#4e) Write an SQL-query to calculate the average area per room size

conn = sqlite3.connect('apartment_database.db') 

cursor = conn.cursor()

cursor.execute('''SELECT rooms, AVG(area)    
               FROM apartments_table    
               GROUP BY rooms''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['Rooms','Avg_Area'])    

conn.close()
df

,Rooms,Avg_Area
0,NaN,NaN
1,1.0,32.092593
2,1.5,35.526316
3,2.0,54.428571
4,2.5,67.317881
5,3.0,68.980392
6,3.5,89.858974
7,4.0,96.129032
8,4.5,114.779661
9,5.0,110.000000


In [60]:
#4f ) 
# Create data base
import json

conn = sqlite3.connect('supermarkets_database.db') 
cursor = conn.cursor()

# Load the JSON data from file
with open('supermarkets.json') as json_file:
    data = json.load(json_file)

# Use json_normalize to flatten the JSON and create a DataFrame
df = pd.json_normalize(data)

# Select columns
df = df[["id", "lat", "lon", "tags.brand", "tags.shop", 
           "tags.addr:city", "tags.addr:street", 
           "tags.addr:housenumber", "tags.addr:postcode"]]

# Rename columns
df = df.rename(columns={"tags.brand":"brand", 
                        "tags.shop":"shop", 
                        "tags.addr:city":"city", 
                        "tags.addr:street":"street", 
                        "tags.addr:housenumber":"housenumber", 
                        "tags.addr:postcode":"postcode"})

# Create db table
cursor.execute('''CREATE TABLE IF NOT EXISTS supermarkets_table (id INT(20),
                                                                 lat DECIMAL(8,2),
                                                                 lon DECIMAL(8,2),
                                                                 brand VARCHAR(50),
                                                                 shop VARCHAR(50),
                                                                 city VARCHAR(100),
                                                                 street VARCHAR(200),
                                                                 housenumber VARCHAR(50),
                                                                 postcode INT(4))''')
# Confirm changes to the table
conn.commit()

# Write data to table
df.to_sql(name = 'supermarkets_table',
          con = conn,
          index = False,
          if_exists = 'replace')

3392

In [61]:
#4d) Write an SQL-query to calculate the average price per room size

# Open a connection to the data base 'apartment_database.db'
conn = sqlite3.connect('apartment_database.db') 

cursor = conn.cursor()

cursor.execute('''SELECT rooms, AVG(price)    
               FROM apartments_table    
               GROUP BY rooms''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['Rooms','Avg_Price'])    

conn.close()
df

,Rooms,Avg_Price
0,NaN,3250.844444
1,1.0,1290.311475
2,1.5,2036.418605
3,2.0,2133.375000
4,2.5,2450.907407
5,3.0,2112.017857
6,3.5,2643.370968
7,4.0,2890.257143
8,4.5,3029.432990
9,5.0,2993.333333


In [62]:
# 4e) 

# Open a connection to the data base 'apartment_database.db'
conn = sqlite3.connect('apartment_database.db') 

# Create cursor object
cursor = conn.cursor()

# Query the SQL-table
cursor.execute('''SELECT rooms, AVG(area)    
               FROM apartments_table    
               GROUP BY rooms''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['Rooms','Avg_Area'])    

# Close connection
conn.close()

# Show results
df

,Rooms,Avg_Area
0,NaN,NaN
1,1.0,32.092593
2,1.5,35.526316
3,2.0,54.428571
4,2.5,67.317881
5,3.0,68.980392
6,3.5,89.858974
7,4.0,96.129032
8,4.5,114.779661
9,5.0,110.000000


## Close db connection (if open)

In [63]:
# Close db connection (if open)
try:
    if check_conn(conn):
        conn.close()
    else:
        pass
except:
    pass

# Status (True = open, False = closed)
print(check_conn(conn))

False


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [64]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.5.0-1025-azure
Datetime: 2024-11-16 13:13:08
Python Version: 3.11.10
-----------------------------------
